In [ ]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import gridspec

In [ ]:
history_txt1 = Path(
    r"1753531432011792_ForArticle001_X50Y100Z50_T3e2_C9.71066e-9_Nt1e10\sim_history.txt"
).read_text(encoding="utf-8")
history_data1 = np.array(
    [
        *map(
            lambda line: [*map(float, line.split(":"))],
            history_txt1.splitlines(),
        )
    ]
)
# print(history_data1)
(
    n_gas_history1,
    n_crystal_history1,
    concentration_history1,
    delta_gibbs_history1,
    energy_change_history1,
    crystal_sx_history1,
    crystal_sy_history1,
    crystal_sz_history1,
    mk_step_history1,
) = history_data1

In [ ]:
history_txt2 = Path(
    r"1753531432559892_ForArticle001_X50Y100Z50_T3e2_C9.71066e-9_Nt1e10\sim_history.txt"
).read_text(encoding="utf-8")
history_data2 = np.array(
    [
        *map(
            lambda line: [*map(float, line.split(":"))],
            history_txt2.splitlines(),
        )
    ]
)
# print(history_data2)
(
    n_gas_history2,
    n_crystal_history2,
    concentration_history2,
    delta_gibbs_history2,
    energy_change_history2,
    crystal_sx_history2,
    crystal_sy_history2,
    crystal_sz_history2,
    mk_step_history2,
) = history_data2

In [ ]:
from dataclasses import dataclass
from functools import partial
from typing import Callable, List, Tuple

import numpy as np
from scipy.optimize import curve_fit
from scipy.optimize import minimize as opt_minimize


# def calculate_r2(y_true: np.ndarray, y_pred: np.ndarray) -> float:
#     """Compute coefficient of determination."""
#     ss_res = np.sum((y_true - y_pred) ** 2)
#     ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
#     return 1 - ss_res / ss_tot


def calculate_r2(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Compute coefficient of determination."""

    if y_true.shape != y_pred.shape:
        raise ValueError("y_true and y_pred must have the same shape.")

    if y_true.size < 2:  # R^2 не определен для менее чем 2 точек
        return np.nan  # Или можно поднять ошибку

    y_mean = np.mean(y_true)

    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - y_mean) ** 2)

    # Обработка случая, когда ss_tot равно нулю (все y_true одинаковы)
    if ss_tot == 0:
        return (
            1.0 if ss_res == 0 else 0.0
        )  # Если все точки одинаковы и модель их точно предсказывает, R^2 = 1

    return 1 - (ss_res / ss_tot)


@dataclass
class ModelSpec:
    name: str
    t1: str
    t2: str
    func: Callable
    init_guess: List[float]
    bounds: List


MODELS = {
    0: {
        "name": "linear",
        "t1": "c0 * x + c1",
        "t2": "{0} * x + {1}",
        "func": lambda x, c0, c1: c0 * x + c1,
        "init_guess": [-0.00014382693355996512, 25930.775363018805],
        "bounds": ((-np.inf, -np.inf), (np.inf, np.inf)),
    },
    1: {
        "name": "exp001",
        "t1": "c0 * EXP[c1 * x] + c2",
        "t2": "{0} * EXP[{1} * x] + {2}",
        "func": lambda x, c0, c1, c2: c0 * np.exp(c1 * x) + c2,
        "init_guess": [1186.911509826299, -4.202561431523349e-05, 25824.243543934055],
        "bounds": ((-np.inf, -np.inf, -np.inf), (np.inf, np.inf, np.inf)),
    },
    2: {
        "name": "exp002",
        "t1": "c0 * EXP[c1 * x]\n + c2 * EXP[c3 * x]\n + c4",
        "t2": "{0} * EXP[{1} * x]\n + {2} * EXP[{3} * x]\n + {4}",
        "func": lambda x, c0, c1, c2, c3, c4: c0 * np.exp(c1 * x)
        + c2 * np.exp(c3 * x)
        + c4,
        "init_guess": [0.09488065437211735, 6.697591186684924e-06, 1194.627363917152, -4.042924858937313e-05, 25810.560463182235],
        "bounds": (
            (-np.inf, -np.inf, -np.inf, -np.inf, -np.inf),
            (np.inf, np.inf, np.inf, np.inf, np.inf),
        ),
    },
    3: {
        "name": "log001",
        "t1": "c1 * LN[x + c2] + c3",
        "t2": "{0} * LN[x + {1}] + {2}",
        "func": lambda x, c1, c2, c3: c1 * np.log(x + c2) + c3,
        "init_guess": [0.01, 1e3, -0.01],
        "bounds": ((-np.inf, 1e-10, -np.inf), (np.inf, np.inf, np.inf)),
    },
    4: {
        "name": "pow001",
        "t1": "c1 * (x ^ c2) + c3",
        "t2": "{0} * (x ^ {1}) + {2}",
        "func": lambda x, c1, c2, c3: c1 * (x**c2) + c3,
        "init_guess": [1e-17, 0.5, -1e-17],
        "bounds": ((-np.inf, -np.inf, -np.inf), (np.inf, np.inf, np.inf)),
    },
}


models = {
    model["name"]: ModelSpec(
        name=model["name"],
        t1=model["t1"],
        t2=model["t2"],
        func=model["func"],
        init_guess=model["init_guess"],
        bounds=model["bounds"],
    )
    for model_id, model in MODELS.items()
    if model_id in [0, 1, 2]
}


def find_best_fits(
    x,
    y,
    methods=("lm", "trf", "dogbox"),
):
    results = []
    for method in methods:
        for spec_name, spec in models.items():
            try:
                popt, _ = curve_fit(
                    spec.func,
                    x,
                    y,
                    p0=spec.init_guess,
                    method=method,
                    maxfev=10_000_000,
                )
                y_pred = spec.func(x, *popt)
                r2 = calculate_r2(y, y_pred)
                results.append((method, spec.name, popt.tolist(), r2))
            except Exception:
                continue

    return results


# func_m = lambda x1, x2: np.sqrt(np.sum(np.power(x1 - x2, 2)))
# func_m = lambda x1, x2: np.sum((x1 - x2) ** 2)
func_m = lambda x1, x2: (1 - calculate_r2(x1, x2)) ** 2

func1_base = lambda popt, x, y, func: func_m(y, func(x, *popt))


# def find_best_fits(x, y):
#     methods = [
#         "Nelder-Mead",
#         "Powell",
#         "CG",
#         "BFGS",
#         "L-BFGS-B",
#         "TNC",
#         "COBYLA",
#         "SLSQP",
#         "trust-constr",
#     ]

#     results = []
#     for method in methods:
#         for spec in models:
#             try:
#                 objective_function = partial(func1_base, x=x, y=y, func=spec.func)

#                 initial_guess = spec.init_guess

#                 bounds = [(None, None) for _ in initial_guess]
#                 # bounds = spec.bounds

#                 result = opt_minimize(
#                     objective_function, initial_guess, bounds=bounds, method=method
#                 )
#                 # print(result)

#                 popt = result.x

#                 y_pred = spec.func(x, *popt)
#                 r2 = calculate_r2(y, y_pred)
#                 results.append((method, spec.name, popt.tolist(), r2))
#             except Exception:
#                 continue

#     return results


if __name__ == "__main__":
    # Example usage
    x_data = np.array(mk_step_history1)  # replace with actual data
    y_data = np.array(n_crystal_history1)

    fits = find_best_fits(x_data, y_data)
    # print(fits)
    fits = sorted(fits, key=lambda tup: tup[3], reverse=True)
    best = {}
    for method, model_name, params, r2 in fits:
        # print(f"Model {model_name} via {method}: R² = {r2:.6f}, params = {params}")
        if model_name not in best:
            best[model_name] = {"m": models[model_name], "params": params, "r2": r2}
            print(f"Model {model_name} via {method}: R² = {r2:.6f}, params = {params}")

In [ ]:
# Model linear via lm: R² = 0.073517, params = [-0.00014382693355996512, 25930.775363018805]
# Model exp001 via lm: R² = 0.949857, params = [1186.9114891439688, -4.2025611322554696e-05, 25824.243542581833]
# Model exp002 via dogbox: R² = 0.964264, params = [0.09485166175483875, 6.697905957541872e-06, 1194.627398868328, -4.042933069443972e-05, 25810.56094542821]

# Model linear via dogbox: R² = 0.544713, params = [-1.2258623942509995e-26, -9.417073230694999e-18]
# Model exp001 via lm: R² = 0.903819, params = [1.1664634891136209e-17, -2.291464085367435e-08, -1.356816919076386e-17]
# Model exp002 via lm: R² = 0.981922, params = [1.0416355587344295e-17, -6.810034732088697e-08, 5.051525077758151e-18, -5.4177598373162016e-09, -1.4601912313332972e-17]

In [ ]:
print(best1["exp001"]["m"].t2.format(*map(lambda v: f"{v:.5e}", best1["exp001"]["params"])))
print(best1["exp002"]["m"].t2.format(*map(lambda v: f"{v:.5e}", best1["exp002"]["params"])))
print()
print(best2["exp001"]["m"].t2.format(*map(lambda v: f"{v:.5e}", best2["exp001"]["params"])))
print(best2["exp002"]["m"].t2.format(*map(lambda v: f"{v:.5e}", best2["exp002"]["params"])))

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

# === 1. Глобальный стиль ===
plt.rcParams.update(
    {
        "font.family": "serif",
        "font.serif": ["Times New Roman"],
        "font.size": 8,
        "axes.labelsize": 8,
        "axes.titlesize": 8,
        "legend.fontsize": 7,
        "xtick.labelsize": 7,
        "ytick.labelsize": 7,
        "lines.linewidth": 0.5,
        "lines.markersize": 0.75,
        "xtick.direction": "in",
        "ytick.direction": "in",
        "axes.grid": True,
        "grid.linestyle": ":",
        "grid.linewidth": 0.4,
    }
)

# === 2. Фигура под журнальную колонку ===
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(3.5, 2), dpi=600)

# === 3. Определяем оттенки серого и маркеры ===
grays = [".8", ".5", ".2"]
markers = ["o", "o", "^"]

# === 4. Первый график ===
datasets = [
    (
        "Data",
        mk_step_history1,
        n_crystal_history1,
        "solid",
        1.5,
        ".75",
    ),
    (
        "exp1 →\nR²: {:.5f}".format(best1["exp001"]["r2"]),
        mk_step_history1,
        best1["exp001"]["m"].func(mk_step_history1, *best1["exp001"]["params"]),
        (0, (5, 2, 0, 2)),
        0.55,
        ".50",
    ),
    (
        "exp2 →\nR²: {:.5f}".format(best1["exp002"]["r2"]),
        mk_step_history1,
        best1["exp002"]["m"].func(mk_step_history1, *best1["exp002"]["params"]),
        (0, (5, 2, 1, 2)),
        0.55,
        ".25",
    ),
]

for label, x, y, ls, lw, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        label=label,
    )


ax1.set_xlabel("Number of $MCS$")
ax1.set_ylabel("Value of $N_{cryst}$")
ax1.legend(loc="best", frameon=True)

# === 5. Второй график ===
datasets = [
    (
        "Data",
        mk_step_history2,
        energy_change_history2,
        "solid",
        1.5,
        ".75",
    ),
    (
        "exp1 →\nR²: {:.5f}".format(best2["exp001"]["r2"]),
        mk_step_history2,
        best2["exp001"]["m"].func(mk_step_history2, *best2["exp001"]["params"]),
        (0, (5, 2, 0, 2)),
        0.55,
        ".50",
    ),
    (
        "exp2 →\nR²: {:.5f}".format(best2["exp002"]["r2"]),
        mk_step_history2,
        best2["exp002"]["m"].func(mk_step_history2, *best2["exp002"]["params"]),
        (0, (5, 2, 1, 2)),
        0.55,
        ".25",
    ),
]

for label, x, y, ls, lw, color in datasets:
    ax2.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        label=label,
    )


ax2.set_xlabel("Number of $MCS$")
ax2.set_ylabel("Surface Energy Change")
ax2.legend(loc="best", frameon=True)


# === 6. Оформление сетки и делений ===
for ax in (ax1, ax2):
    ax.grid(True, which="both", linestyle=":", linewidth=0.4)
    ax.tick_params(length=2, width=0.5)

# === 7. Финализация и сохранение ===
plt.tight_layout()

plt.savefig(
    Path(  #
        r"_materials\article files\figs"
        #
    )
    / "fig_8ab.png",
    format="png",
)
plt.show()
plt.close()

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

# === 1. Глобальный стиль ===
plt.rcParams.update(
    {
        "font.family": "serif",
        "font.serif": ["Times New Roman"],
        "font.size": 8,
        "axes.labelsize": 8,
        "axes.titlesize": 8,
        "legend.fontsize": 7,
        "xtick.labelsize": 7,
        "ytick.labelsize": 7,
        "lines.linewidth": 0.5,
        "lines.markersize": 2.0,
        "xtick.direction": "in",
        "ytick.direction": "in",
        "axes.grid": True,
        "grid.linestyle": ":",
        "grid.linewidth": 0.4,
    }
)

# === 2. Фигура под журнальную колонку ===
fig, ax = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600)

# === 4. Первый график ===
# datasets = [
#     (
#         "Data",
#         mk_step_history1,
#         n_crystal_history1,
#         "solid",
#         1.5,
#         ".8",
#     ),
#     (
#         "exp1 →\nR²: {:.5f}".format(best1["exp001"]["r2"]),
#         mk_step_history1,
#         best1["exp001"]["m"].func(mk_step_history1, *best1["exp001"]["params"]),
#         (0, (5, 2, 0, 2)),
#         0.55,
#         ".5",
#     ),
#     (
#         "exp2 →\nR²: {:.5f}".format(best1["exp002"]["r2"]),
#         mk_step_history1,
#         best1["exp002"]["m"].func(mk_step_history1, *best1["exp002"]["params"]),
#         (0, (5, 2, 1, 2)),
#         0.55,
#         ".2",
#     ),
# ]


# for label, x, y, ls, lw, color in datasets:
#     ax.plot(
#         x,
#         y,
#         linewidth=lw,
#         linestyle=ls,
#         color=color,
#         label=label,
#     )

datasets = [
    ("ΔX", mk_step_history2, crystal_sx_history2, "s", "-", 0.55, ".50"),
    ("ΔY", mk_step_history2, crystal_sy_history2, "o", "-", 0.55, ".25"),
    ("ΔZ", mk_step_history2, crystal_sz_history2, "^", "-", 0.55, ".75"),
]

for label, x, y, marker, ls, lw, color in datasets:
    ax.plot(
        x,
        y,
        marker=marker,
        linewidth=lw,
        linestyle=ls,
        color=color,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=mk_step_history2.size // 10,
        label=label,
    )

ax.set_xlabel("Number of $MCS$")
ax.set_ylabel("Size")
ax.legend(loc="best", frameon=True)

# === 6. Оформление сетки и делений ===
ax.grid(True, which="both", linestyle=":", linewidth=0.4)
ax.tick_params(length=2, width=0.5)

# === 7. Финализация и сохранение ===
plt.tight_layout()

plt.savefig(
    Path(  #
        r"_materials\article files\figs"
        #
    )
    / "fig_8d.png",
    format="png",
)
plt.show()
plt.close()

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 4), dpi=150)
ax.plot(
    energy_change_history,
    color="blue",
    linewidth=1,
    marker="o",
    markersize=2,
    markeredgewidth=0.5,
    markerfacecolor="blue",
    markeredgecolor="black",
    markevery=1,
    # label="Size X",
)

# ax.set_title(f"Axis Z")
# ax.legend(loc="best")
ax.set_xlabel("State ID")
ax.set_ylabel("dE SUM")
ax.grid(True)
# plt.savefig(Path(r"AxisVelocity") / f"AxisX.png")
plt.show()
plt.close()

In [ ]:
history_txt1 = Path(
    r"1753449656486296_Mode2_X50Y100Z50_T3e2_C9.71066e-9_Nt1e11_Pb-1.0\sim_history.txt"
).read_text(encoding="utf-8")
history_data1 = np.array(
    [
        *map(
            lambda line: [*map(float, line.split(":"))],
            history_txt1.splitlines(),
        )
    ]
)
# print(history_data1)
(
    n_gas_history1,
    n_crystal_history1,
    concentration_history1,
    delta_gibbs_history1,
    energy_change_history1,
    crystal_sx_history1,
    crystal_sy_history1,
    crystal_sz_history1,
    mk_step_history1,
) = history_data1

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 4), dpi=150)
ax.plot(
    concentration_history1,
    color="blue",
    linewidth=1,
    marker="o",
    markersize=2,
    markeredgewidth=0.5,
    markerfacecolor="blue",
    markeredgecolor="black",
    markevery=1,
    # label="Size X",
)

# ax.set_title(f"Axis Z")
# ax.legend(loc="best")
ax.set_xlabel("State ID")
# ax.set_ylabel("dE SUM")
ax.grid(True)
# plt.savefig(Path(r"AxisVelocity") / f"AxisX.png")
plt.show()
plt.close()

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 4), dpi=150)
ax.plot(
    crystal_sx_history1,
    color="blue",
    linewidth=1,
    marker="o",
    markersize=2,
    markeredgewidth=0.5,
    markerfacecolor="blue",
    markeredgecolor="black",
    markevery=1,
    # label="Size X",
)
ax.plot(
    crystal_sy_history1,
    color="green",
    linewidth=1,
    marker="o",
    markersize=2,
    markeredgewidth=0.5,
    markerfacecolor="blue",
    markeredgecolor="black",
    markevery=1,
    # label="Size X",
)
ax.plot(
    crystal_sz_history1,
    color="red",
    linewidth=1,
    marker="o",
    markersize=2,
    markeredgewidth=0.5,
    markerfacecolor="blue",
    markeredgecolor="black",
    markevery=1,
    # label="Size X",
)

# ax.set_title(f"Axis Z")
# ax.legend(loc="best")
ax.set_xlabel("State ID")
# ax.set_ylabel("dE SUM")
ax.grid(True)
# plt.savefig(Path(r"AxisVelocity") / f"AxisX.png")
plt.show()
plt.close()

In [ ]:
[
    66 * 20 * 0.41 * 7.38546e-11 * 1.82977e-10,
    20 * 23 * 0.54 * 1.82977e-10 * 2.42724e-10,
    23 * 66 * 0.22 * 2.42724e-10 * 7.38546e-11,
]

In [ ]:
[
    66 * 19 * 0.41 * 7.38546e-11 * 1.82977e-10,
    19 * 22 * 0.54 * 1.82977e-10 * 2.42724e-10,
    22 * 66 * 0.22 * 2.42724e-10 * 7.38546e-11,
]

In [ ]:
[
    54 * 16 * 0.41 * 7.38546e-11 * 1.82977e-10,
    16 * 19 * 0.54 * 1.82977e-10 * 2.42724e-10,
    19 * 54 * 0.22 * 2.42724e-10 * 7.38546e-11,
]